In [2]:
from dotenv import load_dotenv
load_dotenv() 


True

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

c:\PythonProject\potenup_10\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime
query = "오늘 날씨가 어때?"
now = datetime.now()
today = now.strftime('%Y년%m월%d일')

In [5]:
model.invoke(query)

AIMessage(content='죄송하지만 현재 위치나 실시간 날씨 정보를 확인할 수 있는 기능이 없습니다. 오늘 날씨를 알고 싶으시면, 스마트폰의 날씨 앱이나 인터넷 날씨 서비스를 이용해 보세요! 도움이 필요하시면 언제든 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 14, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_c064fdde7c', 'id': 'chatcmpl-CQrCFXvJCSGZQ0iktjTK5ckbmGXcj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a6ca6a2c-b48c-4ca2-bc34-14197c032004-0', usage_metadata={'input_tokens': 14, 'output_tokens': 54, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
import nest_asyncio
import os 
from langchain_mcp_adapters.client import MultiServerMCPClient
from urllib.parse import urlencode
nest_asyncio.apply()


MCP_KEY = os.getenv("MCP_KEY")
if not MCP_KEY:
    print("경고: MCP_KEY 환경 변수가 설정되지 않았습니다. MCP 호출이 실패할 수 있습니다.")

# 3. 클라이언트 초기화 및 전역 변수 설정 함수

base_url = "https://server.smithery.ai/@sunhye/mcp-korea-weather/mcp"
params = {
    "api_key": MCP_KEY, # 환경 변수에서 불러온 키 사용
    "profile": "usual-reindeer-MZSQQr" # 예시에 있던 프로필 값 (필요시 변경)
}
final_url = f"{base_url}?{urlencode(params)}"
print(f"접속할 최종 URL: {final_url}")
client = MultiServerMCPClient(
    {
        "weather": {
            # 4. 새로 만든 최종 URL을 사용합니다.
            "url": final_url,
            "transport": "streamable_http",
        }
    }
)


접속할 최종 URL: https://server.smithery.ai/@sunhye/mcp-korea-weather/mcp?api_key=6bf43259-dc31-4143-a477-2dd5433e8adb&profile=usual-reindeer-MZSQQr


In [10]:
# nest_asyncio.apply()
# 사용 가능한 도구 목록을 비동기적으로 가져옵니다.
tools = await client.get_tools()

# 첫 번째 도구를 비동기적으로 실행하고 결과를 기다립니다.
# 입력값은 딕셔너리 형태로 정확하게 전달합니다.

# 서울의 위도 경도
seoul_latitude = "37.5665"
seoul_longitude = "126.9780"
result = await tools[1].ainvoke(
    {
        "location": f"""오늘 날짜:{today} 
                                             질문:{query}""",
        "latitude": seoul_latitude,
        "longitude": seoul_longitude,
    }
)

# 최종 결과를 확인합니다.
print(result)

(37.5665, 126.9780) nx=60, ny=127 / 기준 20251015 1730
20251015-1800 → 21°C 흐림 없음 강수확률
20251015-1900 → 20°C 흐림 없음 강수확률
20251015-2000 → 19°C 흐림 강수확률


In [ ]:
# 5. MCP 클라이언트를 사용하여 실제 요청을 보내는 비동기 함수 정의
async def get_real_weather_async(query: str):
    global client
    
    try:
        # 실제 MCP 호출 및 결과 반환
        result = await client.get_tools()
        print("[MCP 응답] 데이터 수신 완료.")
        return result
    except Exception as e:
        print(f"[MCP 오류] 요청 실패: {e}")
        return {"error": f"MCP 호출 중 오류 발생: {e}"}

In [ ]:
import asyncio
asyncio.run(get_real_weather_async(query))

In [ ]:
from datetime import datetime

model.invoke(f"""

[역할]
당신은 서울 날씨를 알려주는 봇 입니다.
오늘 날짜: {today} 

[목표]
오늘 날짜를 기반으로 서울 날씨를 상세하게 알려주세요

[제약 조건]
- 당일날 새벽의 날씨는 빼주세요 오전 ~ 밤 사이의 날씨이어야 합니다.
- **절대 정보를 지어내지 마세요.**

[출력 형식]
- 비, 눈, 맑음 등 확실한 상태를 알려주세요
- 오늘의 온도를 아침, 점심, 저녁으로 모두 파악해주세요.

질문: 오늘 날씨가 어때?
""")
